In [1]:
# !pip install geopy pandas numpy shapely

In [2]:
import pandas as pd
import json
import numpy as np
from geopy import distance
from shapely.geometry import Point

# Load files

In [3]:
batch = pd.read_csv("../data/batch.csv")
batch

,id,spire_update_statement,vessel_mmsi,position_latitude,position_longitude,position_timestamp,position_update_timestamp
0,1,2024-04-06 13:00:00,123456789,45.578946,2.345674,2024-04-06 12:57:34,2024-04-06 12:59:17
1,2,2024-04-06 13:00:00,987654321,37.454125,5.541247,2024-04-06 12:53:34,2024-04-06 12:57:00
2,3,2024-04-06 13:00:00,123789456,24.454815,10.458147,2024-04-06 12:51:34,2024-04-06 12:54:13
3,4,2024-04-06 13:00:00,456789123,17.477546,6.124655,2024-04-06 12:50:34,2024-04-06 12:58:18
4,5,2024-04-06 13:00:00,654321987,-3.134575,-5.484315,2024-04-06 12:44:34,2024-04-06 12:59:39
5,6,2024-04-06 13:00:00,321987654,-24.223149,17.464132,2024-04-06 12:56:34,2024-04-06 12:59:13
6,7,2024-04-06 13:00:00,321654987,48.452135,11.138797,2024-04-06 12:52:17,2024-04-06 12:55:01
7,8,2024-04-06 13:00:00,456123789,44.234659,3.134643,2024-04-06 12:43:34,2024-04-06 12:56:24
8,9,2024-04-06 13:00:00,789456123,-48.465131,17.142316,2024-04-06 12:55:01,2024-04-06 12:56:39
9,10,2024-04-06 13:00:00,789123456,0.131748,-0.013547,2024-04-06 12:57:33,2024-04-06 12:57:47


In [4]:
excursions = pd.read_csv("../data/excursions.csv")
excursions

,vessel_id,mmsi,id,arrival_at
0,7,123456789,34,NaN
1,13,987654321,11,NaN
2,11,123789456,16,NaN
3,59,456789123,87,NaN
4,546,321987654,37,NaN
5,87,321654987,45,NaN
6,65,789456123,46,NaN
7,66,789123456,89,NaN


In [5]:
last_segment = pd.read_csv("../data/last_segment.csv")
last_segment

,excursion_id,timestamp_end,last_vessel_segment,mmsi,end_position
0,34,2024-04-06 12:44:34,1,123456789,"[45.57894563,2.34567410]"
1,11,2024-04-06 12:43:34,1,987654321,"[37.35412456,5.52124685]"
2,16,2024-04-06 12:44:34,1,123789456,"[24.47481487,10.488146567]"
3,87,2024-04-06 12:37:34,1,456789123,"[17.97754648,6.22465498]"
4,104,2024-04-06 12:44:34,1,654321987,"[-3.13457514,-5.48431548]"
5,37,2024-04-06 12:44:34,1,321987654,"[-24.12314854,17.36413158]"
6,45,2024-04-06 12:38:17,1,321654987,"[48.55213468,11.83879748]"
7,5,2024-04-06 12:43:34,1,456123789,"[44.23465853,3.13464328]"
8,46,2024-04-06 12:40:01,1,789456123,"[-48.26513147,17.44231649]"
9,89,2024-04-06 12:41:33,1,789123456,"[0.14174789,-0.05354678]"


# Step-by-step ETL

In [6]:
# Step 1: load SPIRE batch
batch = pd.read_csv("../data/batch.csv")
batch_columns = list(batch.columns)

## Parse dates
for col in ["spire_update_statement", "position_timestamp", "position_update_timestamp"]:
    batch[col] = pd.to_datetime(batch[col])

In [7]:
# Step 2: load excursion from fct_excursion where date_arrival IS NULL
excursions = pd.read_csv("../data/excursions.csv")
excursions = excursions.loc[excursions["arrival_at"].isna()].copy()

In [8]:
# Step 3: load last_segment where last_vessel_segment == 1
last_segment = pd.read_csv("../data/last_segment.csv")
last_segment = last_segment.loc[last_segment["last_vessel_segment"] == 1]
last_segment["end_position"] = last_segment["end_position"].apply(json.loads) # Load list of coordinates inside of str
last_segment["end_position"] = last_segment["end_position"].apply(lambda pos: (pos[1], pos[0])) # reverse (lat, lon) to (lon, lat)
last_segment["timestamp_end"] = pd.to_datetime(last_segment["timestamp_end"]) # convert to datetime

In [9]:
# Step 4: merge last_segment with excursions
excursions = excursions.merge(last_segment, left_on=["id", "mmsi"], right_on=["excursion_id", "mmsi"])

In [10]:
# Step 5: merge batch with excursions on mmsi
batch = batch.merge(excursions, how="left", left_on="vessel_mmsi", right_on="mmsi", suffixes=["", "_excursion"])
# Est-ce normal que vessel_id ne soit pas dans excursions

In [11]:
# Step 6: compute speed between last and current position

In [12]:
# Step 6.1. Compute distance in km between last and current position 
def get_distance(current_position: tuple, last_position: tuple):
    if isinstance(last_position, float) and np.isnan(last_position):
        return np.nan
    
    return distance.geodesic(current_position, last_position).km

batch["distance_since_last_position"] = batch.apply(
    lambda row: get_distance((row["position_longitude"], row["position_latitude"]), row["end_position"]),
    axis=1
)

In [13]:
# Step 6.2. Compute time in hours between last and current position 
batch["time_since_last_position"] = batch["position_timestamp"] - batch["timestamp_end"] # Quelle est la date à prendre ?
batch["hours_since_last_position"] = batch["time_since_last_position"].dt.seconds / 3600

In [14]:
# Step 6.3. Compute speed: speed = distance / time
batch["speed_since_last_position"] = batch["distance_since_last_position"] / batch["hours_since_last_position"]

In [15]:
# Step 7: apply to_keep flag: keep only positions WHERE:
# - row["excursion_id"] is not NULL, i.e. it was merged with an excursion where date_arrival IS NULL
# - OR speed is not close to 0, i.e. vessel moved significantly since last position
batch["to_keep"] = batch.apply(
    lambda row: np.isnan(row["excursion_id"]) is False or row["speed_since_last_position"] > 0.1,
    axis=1
)

In [16]:
# Step 8: filter unflagged rows to insert to DB
batch = batch.loc[batch["to_keep"] == True, batch_columns].copy()
batch

,id,spire_update_statement,vessel_mmsi,position_latitude,position_longitude,position_timestamp,position_update_timestamp
1,2,2024-04-06 13:00:00,987654321,37.454125,5.541247,2024-04-06 12:53:34,2024-04-06 12:57:00
2,3,2024-04-06 13:00:00,123789456,24.454815,10.458147,2024-04-06 12:51:34,2024-04-06 12:54:13
3,4,2024-04-06 13:00:00,456789123,17.477546,6.124655,2024-04-06 12:50:34,2024-04-06 12:58:18
5,6,2024-04-06 13:00:00,321987654,-24.223149,17.464132,2024-04-06 12:56:34,2024-04-06 12:59:13
6,7,2024-04-06 13:00:00,321654987,48.452135,11.138797,2024-04-06 12:52:17,2024-04-06 12:55:01
8,9,2024-04-06 13:00:00,789456123,-48.465131,17.142316,2024-04-06 12:55:01,2024-04-06 12:56:39
9,10,2024-04-06 13:00:00,789123456,0.131748,-0.013547,2024-04-06 12:57:33,2024-04-06 12:57:47
